In [4]:
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [5]:
import os

In [6]:
from sklearn.metrics import roc_auc_score,accuracy_score,precision_score

In [7]:
os.chdir('../output_data')

In [8]:
df = pd.read_csv('train_v04.csv',header=0)

In [9]:
df.shape

(85799, 216)

In [10]:
x = df.drop(['lift_ind'],axis=1)

In [11]:
y = np.array(df['lift_ind'])

In [9]:
model = GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=8,
              max_features='log2', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=0.1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200,
              presort='auto', random_state=42, subsample=0.8, verbose=0,
              warm_start=False)

In [10]:
model.fit(x,y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=8,
              max_features='log2', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=0.1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200,
              presort='auto', random_state=42, subsample=0.8, verbose=0,
              warm_start=False)

In [11]:
os.chdir('../Models')

In [12]:
pickle.dump(model,open('GBM_full.pickle','wb'))

In [13]:
y_prob = model.predict_proba(x)

In [14]:
gini_train = roc_auc_score(y,y_prob[:,1])

In [15]:
gini_train

0.6529784222607431

In [16]:
y_pred = model.predict(x)
accuracy_score(y,y_pred)

0.6069418058485531

In [18]:
precision_score(y_true=y,y_pred=y_pred)

0.6181965881397238

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [13]:
scale = StandardScaler()

In [14]:
scale.fit(x)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [15]:
scaled_x = scale.transform(x)

In [16]:
scaled_x.shape

(85799, 215)

In [17]:
pca = PCA(n_components=215)
pca.fit(scaled_x)

PCA(copy=True, iterated_power='auto', n_components=215, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [18]:
pca_explained = pca.explained_variance_ratio_.cumsum()

In [19]:
n_components = 0
for i in range(216):
    n_components+=1
    if pca_explained[i] >= .99:
        break

In [20]:
n_components

101

In [21]:
pca_final = PCA(n_components=n_components)
pca_final.fit(scaled_x)
pca_x = pca_final.transform(scaled_x)

In [22]:
os.chdir('../pickles/')

In [23]:
pickle.dump(scale,open('scale.pickle','wb'))
pickle.dump(pca_final,open('pca.pickle','wb'))

In [24]:
os.chdir('../Models/')

In [25]:
model = GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=8,
              max_features='log2', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=0.1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200,
              presort='auto', random_state=42, subsample=0.8, verbose=0,
              warm_start=False)

In [26]:
model.fit(pca_x,y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=8,
              max_features='log2', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=0.1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200,
              presort='auto', random_state=42, subsample=0.8, verbose=0,
              warm_start=False)

In [27]:
pickle.dump(model,open('GBM_pca.pickle','wb'))

In [28]:
y_prob = model.predict_proba(pca_x)

In [29]:
roc_auc_score(y,y_prob[:,1])

0.6387122330551036

In [30]:
y_pred = model.predict(pca_x)
accuracy_score(y,y_pred)

0.5978391356542617

In [31]:
model = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=0.06, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=True, random_state=42, verbose=0, warm_start=False)

In [32]:
model.fit(pca_x,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=0.06, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=True, random_state=42, verbose=0, warm_start=False)

In [33]:
pickle.dump(model,open('RF_pca.pickle','wb'))

In [34]:
y_prob = model.predict_proba(pca_x)

In [35]:
roc_auc_score(y,y_prob[:,1])

0.5990725496578736

In [36]:
y_pred = model.predict(pca_x)
accuracy_score(y,y_pred)

0.5289339036585508

In [37]:
precision_score(y_true=y,y_pred=y_pred)

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.0